<img src="./img/CCIT_Logo.png" alt="CCit Logo" width=400>

# 03 - Simple Power Analysis (SPA) 🕵️

## ⚠️ Prerequisites

Hol' up - before continuing, ensure you have done the following:

* ☑ Read the introduction to Side-Channel Analysis in the [02-Intro to SCA](./02-Intro%20to%20SCA.ipynb) notebook.

### 📑 Summary

*In this lab, you'll learn what is Simple Power Analysis, AKA **SPA**, and how to carry out such an attack. We will practice with the remote ChipWhisperer board, having a look at the firmware running in the target microcontroller. We will try to reverse engineer it just by looking at the power trace collected.*

### 💬 Learning outcomes
* What is SPA
* How to carry out such an attack
* How to do some simple reverse engineering on the target code

## 📑 Table of Contents
1. [What is Simple Power Analysis? ❓](#spa)
2. [Let's try it!](#try)
    1. [Connecting to your ChipWhisperer Nano board](#connecting)
        1. [Configure the scope and the capture request](#config)
        2. [Launch a trace capture](#capture)
    2. [A closer look to the data collected](#look)
    3. [Plotting a power trace](#plotting)
    4. [[Optional] If you look long enough into a trace the trace begins to look back through you](#abyss)
    5. [[Optional] We need to go deeper - How ChipWhisperer's sampling works](#deeper)
    6. [Looking at the big picture](#picture)
    7. [The devil is in the details - Understanding the algorithm](#devil)
3. [Conclusions](#conclusions)

#### Before starting, I suggest you running the following. This will horizontally expand your workspace. This will help you better visualize the plots we are going to create.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display("text/html", "<style>.container { width:100% !important; }</style>")

## 1) What is Simple Power Analysis? ❓ <a class="anchor" id="spa"></a>
Let me recall what I introduced you in the previous notebook:

1. [Simple Power Analysis (SPA)](https://en.wikipedia.org/wiki/Power_analysis#Simple_power_analysis)

> In this first attack, the goal of the attacker is to (but not limited to) understand *what is the operation* the microprocessor is performing. The attack is carried out manually: Eve visually inspects the graph of the power consumption and tries to interpret it. Pattern recognition tools and machine learning can be used if the attacker already knows what to look for. Most of the time, a single power consumption trace is sufficient (meaning Eve can succeed even with a single attempt).

> Is there any spikes in the current consumption of the device? Is there any repetitive pattern in the graph? **What useful information can we gain about the code executed by the microprocessor?**

Now, let me also add the follwing:

> Simple Power Analysis is a preparatory step for Differential/Correlation Power Analysis. Most of the times, SPA is not enough powerful to recover the secret key. However, as we will see in the next notebooks, it is extremely important to know where we shall expect the key to be handled, by looking at *the code flow executed by the microprocessor*. Once this information found, we can properly tailor DPA and CPA so to increase our chances to completely retrieve the key.

> SPA can also be used to reverse engineer a piece of code. With good-enough instruments, such as a ChipWhisperer, and some luck, an attacker can recognize the presence of branches in the code flow. Finally, with enough clues and some good expertise, Eve can go as far as to understand the single assembly instructions.

So, it is called "Simple", but can also be extremely powerful in certain circumstances!

## 2) Let's try it! <a class="anchor" id="try"></a>
Enough with the theory, let's capture our first trace! What are we going to do?

1. Leveraging the CyberChallenge Python library, let's request a trace capture operation to the ChipWhisper board connected to the server;
2. The server will reply sending you the numeric data representing the captured trace;
3. Once a reply obtained, we can forget about the server and analyze the data we got;
4. Using matplotlib, we plot the data and graphically analyze our trace.

> ⚠️ **DISCLAIMER: the code to complete our first connection and capture request is given below, just fill in the blanks (user token).**

### 2.1) Connecting to your ChipWhisperer-Nano board <a class="anchor" id="connecting"></a>
🛑 **STOP! Make sure you have retrieved your personal user token before running the code below! Also, remember that every request is linked to your personal account, so make sure not to disclose your token to your colleagues!**

#### 2.1.1) How does a trace capture work form ChipWhisperer's perspective?

> ℹ️ Each power trace refers to a single execution of the microcontroller code. Remember: you are observing the power consumption of the target device while it's operating. So, if you ask 100 traces, ChipWhisperer will actually launch 100 times the same operation, meaning it will reset the target microcontroller 100 times. At each reset, the microcontroller will start its code from scratch, and ChipWhisperer will record the power consumption trace produced by that specific execution.

> ℹ️ If this sounds time-consuming, don't worry, ChipWhisperer is actually incredibly fast in resetting, capturing the trace and resetting again the target microcontroller.

In pseudocode, ChipWisperer will fulfill your capture request like this:

```python
    configure_capture_probe(scope_config)
    traces_list = []
    # ChipWhisperer is now configured and ready
    for i in range(0, capture_config.num_traces):
        arm_capture_probe()
        reset_target()
        while !triggered_by_target():
            pass # continue looping
        # CW has been triggered, the target is running! Starting recording trace
        # A trace is an array of samples!
        # By default we capture 5000 samples for each trace, no need to change that value
        for j in range(0, scope_config.num_samples):
            trace_data.append(collect_sample())
        traces_list.append(trace_data)

    # Once all the traces captured, send the entire list back to the user
    send_data_to_user(traces_list)
```

Now, let's import the CyberChallenge library:

In [ ]:
from cyberchallenge_client import ccclient

#### 2.1.2) Configure the scope and the capture request <a class="anchor" id="config"></a>
1. Every capture request you send to the server MUST be properly described by a capture configuration. For instance, how many power traces do you want to capture?

> To do so, you need to modify the `ccclient.default_capture_config` dict. For instance, you can change the number of traces you want to capture (by default, `num_traces` is set to `100`):

```python
    ccclient.default_capture_config["num_traces"] = 200
```

> ⚠️ Since requesting an absurd amount of traces can be considered as a form of DDoS, every capture packet you send that requests a number of traces higher than our thresholds is automatically discarded. Don't worry, it can happen, you won't get reported for this. Unless you find a way to circumvent our checks 😌

2. You can configure some parameters useful for the ChipWhipserer scope board (i.e. the board that collects the power trace), by modifying the `ccclient.default_scope_config` dict.

> ⚠️ For this course, you will never need to tweak these parameters, so leave them as they are originally described in `ccclient.default_scope_config`.

Let's configure our capture request in order to ask just 1 (one) single trace. With SPA even a single trace is sufficient to gather tons of information.

In [ ]:
ccclient.default_capture_config["num_traces"] = 1

Now let's create a `connection` object, such object wraps all the technical information needed to connect to the server. 

The `URL`, `PORT` and `YOUR_TOKEN` information are provided to you in your cyberchallenge.it personal portal. Don't change them, of course.

#### 2.1.3) Launch a trace capture <a class="anchor" id="capture"></a>

In [ ]:
connection = ccclient.Utility(str(URL), int(PORT), str(YOUR_TOKEN))

Before launching our capture request, there's still two parameters to configure:

1. **The codename** of the firmware to be flashed and executed by the target microcontroller. 

> ⚠️ It is important to specify the correct firmware codename, otherwise you will capture gibberish data and cannot proceed further! The codename is specified in each notebook, you don't need to change it.

2. **The timeout**, i.e. the number of seconds to wait before giving up with the request.

> ⚠️ I suggest you inserting a timeout of at least 60 seconds. Don't worry, in many cases your request will be fulfilled in a shorter time. However, please be aware: **if you experience many consequent timeout errors, you probably need to increase your timeout value and have a little more patience.**

In [ ]:
(state, project) = connection.capture_request("firestarter", 60, ccclient.default_capture_config)
print(f"State: {state}")

If the above command completed successfully, you should have the variable `state` set to `True`. If it is `False`, run again the above snippet, because something went wrong.

The `project` object collects all the captured traces, in our case just one trace. Each trace is represented by a [`Trace` object](https://chipwhisperer.readthedocs.io/en/latest/api.html#trace).

### 2.2) A closer look to the data collected <a class="anchor" id="look"></a>

Let's now print the data content for the first trace. Run the following.

In [ ]:
# The Project object: an "improved" list containing all the Trace objects requested (only one Trace in our case)
print(f"Project: {project}\n") 

# Trace: an object containing the plaintext sent to the device, the ciphertext obtained as output and the "wave",
# i.e. the power trace collected by the ChipWhisperer scope board during the device cipher activity
print(f"Trace captured: {project.traces[0]}\n")

# Wave: a list collecting all the power samples captured by the CW scope board. 
# By default, each wave contains 5000 samples
print(f"Wave: {project.traces[0].wave}")

# Textin: a bytearray collecting the 16 bytes of the plaintext
print(f"Textin: {project.traces[0].textin}")

# Textout: a bytearray collecting the 16 bytes of the ciphertext
print(f"Textout: {project.traces[0].textout}")

### 2.3) Plotting a power trace <a class="anchor" id="plotting"></a>

Let's now plot the wave we just obtained!

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.figure(figsize=[8,6])
plt.plot(project.traces[0].wave, label="Power trace")
plt.grid(visible=True, which='major', axis='both', alpha=0.2)

plt.show()

You should get something similar to this:

<img id="SPA_firs_trace" src="./img/03-Simple Power Analysis/SPA_1.png" alt="SPA, First trace" width=600>

What you're looking at, ladies and gentlemen, is your first Simple Power Analysis trace!

On the `x` axis you have all the 5000 samples the ChipWhisperer (CW) board captured for you. The absolute value of the `y` axis is, simply put, insignificant, as the current consumption (represented as a voltage drop across the shunt resistor) is quantized by CW's internal ADC, amplified, scaled etc.

> ℹ️ The absolute value on the `y` axis is not what we are looking for! We don't need to know if a certain sample has value 0.3 or 0.2 on the `y` axis. All the information we need is actually carried by the "relative" fluctuations (spikes) from one sample to the other.

### 2.4) [Optional] If you look long enough into a trace the trace begins to look back through you <a class="anchor" id="abyss"></a>

The `%matplotlib notebook` magic allows you to interact with the power trace you just captured. For instance, you can zoom in so much you can discern one sample from the other! In the following image I was even able to annotate 11 different sampling points, from the 560th to the 570th. You can tell the exact value of these 11 sampling point by reading them from the captured wave!

> ⚠️ The annotated image below may not reflect the exact values you will get by executing the following command. This is normal, as your device may be different from the one I used when writing this notebook, hence there may be differences deep down at the silicon level. Moreover, the ambient temperature may be different etc... All this parameters inevitably affect the absolute value of the samples on the `y` axis! If you want to see the values as I did, simply zoom in in the graph you plotted.

In [ ]:
print(project.traces[0].wave[560:571])

<img src="./img/03-Simple Power Analysis/SPA_1_samples.png" alt="SPA, samples annotation" width=500>

### 2.5) [Optional] We need to go deeper - How ChipWhisperer's sampling works <a class="anchor" id="deeper"></a>

The previous picture inherently shows how powerful ChipWhisperer can be. Why? Let's demonstrate it.

It should be clear (if you previously read the documentation!) that the ChipWhisperer Nano board we are using is the union of two boards, the target board, hosting the microcontroller under attack, and the main capture/measurement board, which observes the target microcontroller and records power traces from it.
**The strength of this solution lies on the fact that the two boards operate with two clock signals that are basically "locked" together.** 

> ℹ️ The important thing to retain is that **the two clock signals maintain a constant phase and frequency relationship over time**

* If the above constraints are met, we say we operate in **synchronous sampling** conditions.
* If the above constraints ARE NOT met, we say we operate in **asynchronous sampling** conditions.

> ℹ️ Having a **variable phase** among two clocks disrupts the efficacy of attacks like DPA and CPA (more on them in the following notebooks). Indeed, having the capture clock running freely from the target one means that the captured traces are all misaligned with each other (meaning that sample \#0 from trace \#0 may come slightly early (or late) than the same sample in trace \#1).

> ℹ️ Similarly, a **variable frequency** of the capture clock makes the two signals "running away" from each other (i.e. *drifting*). As a consequence, all the captured traces lose their temporal reference both internally (traces get squelched/stretched) and from each other (like in the case of a variable phase)

I hope the following image can help you better the difference among synchronous and asynchronous sampling:

<img src="./img/03-Simple Power Analysis/clocks.png" alt="Clocks difference" width=1200>

ChipWhisperer uses synchronous sampling: the capture board operates at a frequency which is a multiple of the clock frequency the target board operates at. Let's assume the frequency of the capture clock signal is exactly 4 times the clock frequency of the target board. This means that the capture board can record 4 different samples while the target board is completing one clock period. 

Playing with the multiplier (which is 4 in this example) allows to achieve a finer sampling granularity. In simpler words, the higher the multiplier the higher the details you can see in a power trace, but also the higher the number of sampling points you have to store in memory and later compute!

See the following image, where the multiplier among the target clock and the capture clock is 4:

<img src="./img/03-Simple Power Analysis/capture_clock.png" alt="Capture Clock v. Target Clock" width=300>

As you can see from the previous picture, having the same clock reference, even though the clock frequencies are not originally the same, is incredibly powerful!

**Why? Because in most micropocessors, a rising edge may entail the execution of a new assembly instruction!**

> ⚠️ The above sentence should have a BIG **"IT DEPENDS ON THE PROCESOR ARCHITECTURE!"** in front of it.

> ℹ️ For instance, complex microprocessor architectures may leverage also the falling edge, and/or execute different operations depending on the clock edge. Similarly, many assembly instructions may require more than one clock cycle to complete. Moreover, many microprocessors leverage [pipelining](https://wikiless.org/wiki/Instruction_pipelining?lang=en), meaning they can start computing one instruction while they are finishing computing the previous one!

> Such complexity is way beyond our scope, we won't delve deep into details anymore than that.

What is important to retain is that, knowing the architecture of the microprocessor under attack, Eve can get to the point of discerning each assembly instruction, and even reverse engineer the target code!

**Another important reason to have the same clock reference**

This will be better explained in the following tutorial. For now, it is important to you to know that having the same reference among the two clocks (capture board and target board) implies that all the captured traces will have the same clock reference! This means that the samples of the first trace will have been captured more or less at the same instant in time as the same samples of the second trace, even though they are two different executions of the same algorithm! **As we will see, this is extremely important and leads to a faster retrieval of the secret key.**

> ℹ️ Remember that a trace corresponds to a single execution of the targe microprocessor. Collecting 100 traces means we are launching the same operation 100 different times!

### 2.6) Looking at the big picture <a class="anchor" id="picture"></a>

We just explored the details of our power trace by zooming in as much as possible. Now we want to focus on the big picture, looking for repetitions and patterns that might give us additional information about the operation performed by the target microprocessor.

So let us zoom out as much as possible in [the first plot we made from our power trace](#SPA_firs_trace), can you find some repeating patterns?

Indeed, we can recognize 4½ repeating blocks! However, if you look carefully, it is hard to tell what is the point in which a pattern starts and the one in which it ends. For instance, in the following image, do you think the repeating blocks are all based on `Pattern1` or on `Pattern 2`?

<img src="./img/03-Simple Power Analysis/pattern_recognition.png" alt="SPA, pattern recognition" width=600>

Let's brainstorm it.

1) If all the repeating blocks were based on `Pattern 1`, how do you explain the purple `???` block in front of it? Can it be explained by tagging it as "microcontroller/code initialization"? If yes, what about the remaining spikes on the left of the purple block? Are those the "initialization of the initialization"?

2) On the other hand, what if the purple block was indeed a sub-pattern of `Pattern 2`? And what if the spikes on its left were actually the first "microcontroller/code initialization"?

Which of the two rationales seems the most plausible?

<details>
  <summary> 🗨️ Spoiler (Click me)</summary>
  
The correct pattern is `Pattern 2`.
  
</details>

### 2.7) The devil is in the details - Understanding the algorithm <a class="anchor" id="devil"></a>

Now that we are fairly confident in our assumption, let's enlarge the plot so that the pattern is completely contained in it. Let's also add some additional samples both on the left and on the right, so to have a little bit more context. In the following image, the pattern `Pattern 2` we isolated previusly starts more or less at sample 200 and terminates at sample 1200.

<img src="./img/03-Simple Power Analysis/pattern.png" alt="SPA, pattern" width=600>

Let's partition it into sub-patterns, or groups as I called them here:

> ℹ️ You can use the excellent [draw.io](draw.io) tool to annotate traces, it is extremely useful for SPA

The combination `Group 1` + `Group 2` + `Group 3` + `Group 4` is what composed the `Pattern 2` in the previous pictures.

<img id="pattern_annotated" src="./img/03-Simple Power Analysis/pattern_annotated.png" alt="SPA, pattern annotated" width=600>

It should now be clear that our microcontroller under attack is performing, in the totality of the 5000 samples collected, the same operation more or less 4 times and a half, [see the previous graph](#SPA_firs_trace).

But what kind of operation is it? 

> ℹ️ It's extremely difficult for an inexperienced attacker like you to find it out by yourself: **what we are looking at the `AES-128` algorithm.**

> ⚠️ I need you to stop and be sincere with yourself, do you remember how `AES-128` works? Shall you need to revise some previous lectures? What about refreshing your memories by reading the related [Wikipedia article](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard#High-level_description_of_the_algorithm)? We are now going to delve into the details of an `AES` implementation, so refreshing the topic may be useful.

Let's start from the beginning of the trace, shall we? If we exclude the first 200 samples (more or less) on the left, the first batch of spikes we encounter is what we call `Group 1`, referred in purple in the above image.

This very first appearance of `Group 1` refers to the so-called "key whitening" operation, i.e. the initial `AddRoundKey`step, the one done before even starting the first round of `AES`. From here on, groups `2`, `3`, `4` and `1` (the second appearance of `Group 1`) refer to the very first round of the `AES-128` algorithm.

> ℹ️ A "round" is a set of elementary criptographic operations. An entire encryption/decryption operation may require multiple repetitions of a round. For instance, an encryption/decryption operation in `AES-128` requires 10 rounds, i.e. 10 repetitions of the same round. `AES-192` requires 12 rounds, `AES-256` requires 14 rounds.

> ℹ️ In the case of `AES`, every round is composed by these elementary cryptographic operations: `SubBytes`, `ShiftRows`, `MixColumns` and `AddRoundKey`, in this exact order.

#### 2.7.1) Associating each `Group` to an AES operation

Starting from the *key whitening* operation, i.e. the first appearence of `Group 1` purple block, let's analyze one by one all the colored blocks [annotated in the previous picture](#pattern_annotated).

On the left you will see the annotation of the trace I personally captured, on the right you'll see the corresponding piece of code, executed by the target STM32 microcontroller under attack.

##### Group 1 - `AddRoundKey` operation

The `AddRoundKey` operation simply consists of a `XOR` among a cell of the current `State` matrix and the corresponing cell in the `RoundKey` matrix. As you can see from the image (you can also experiment this by yourself), we can clearly distinguish 16 sets of peaks, corresponding to the 16 repetititions of the `XOR` operation.

Moreover, and that's pretty cool, you can notice how sets of peaks that are multiple of 4 (highlighted in the image) actually include not 2, but 3 peaks each. We can be pretty confident that this third peak corresponds to the code jump that stops the inner loop (the one involving the `j` variable) and goes back to the increment of variable `i` (something that is done only every 4 inner loops).

<img id="AddRoundKey_complete" src="./img/03-Simple Power Analysis/AddRoundKey_complete.png" alt="AddRoundKey Complete" width=1200>

##### Group 2 - `SubBytes` operation

Similarly to the previous image, the `SubBytes` step shows 16 different iterations of the same operation: each single `State` cell is swapped with the corresponding value in the `S-Box` (basically, a Look-Up Table (LUT)).

> ℹ️ If you compare the absolute value of the peaks among the `AddRoundKey` and the `SubBytes` operations you will notice that the spikes of the latter group achieve higher peaks. This is probably due to the fact that the `SBox` matrix is declared as `const`, hence stored in ROM and not in RAM. Hence more energy is needed to move the data.

<img id="SubBytes_complete" src="./img/03-Simple Power Analysis/SubBytes_complete.png" alt="SubBytes Complete" width=1200>

##### Group 3 - `ShiftRows` operation

The `ShiftRows` step can be heavily optimized by the compiler, hence it is quite hard to identify. The C code is extremely simple and verbose, we can expect the related assembly code to be much more "compact" than that.

<img id="ShiftRows_complete" src="./img/03-Simple Power Analysis/ShiftRows_complete.png" alt="ShiftRows Complete" width=1200>

##### Group 4 - `MixColumns` operation

Here again, it is quite simple to find the 4 ripetitions done by this `for` loop. The far right side of the plot going from sample 1200 to sample 1300ish, the one with the black marker on top, is the second `AddRoundKey` operation.

<img id="MixColumns_complete" src="./img/03-Simple Power Analysis/MixColumns_complete.png" alt="MixColumns Complete" width=1200>

## 3) Conclusions! <a class="anchor" id="conclusions"></a>
This marks the end of this tutorial, well done! You have now all the tools necessary to master Simple Power Analysis. If you haven't yet, I suggest you reading the above sections marked as `[Optional]`.

Completing them will help you understand future challenges presented in the next tutorials.

4. [[Optional] If you look long enough into a trace the trace begins to look back through you](#abyss)
5. [[Optional] We need to go deeper - How ChipWhisperer's sampling works](#deeper)

--- 
Do you have suggestions, doubts or simply need to reach me?
* 🐦 Twitter: `[at]mrcuve0`
* 💬 Discord: `Mrcuve0#4179`
* 💻 GitHub:  `Mrcuve0`
* 📬 Email: `mrcuve0 [at] posteo [d.ot] net`
* 💼 LinkedIn:
> Please, send me a message via the previous methods before asking for my LinkedIn

---
Copyright ©️ [CINI - Cybersecurity National Lab](https://cybersecnatlab.it/) - Torino, 2022.

> This material is a derivative work of [NewAE's official Jupyter Notebooks](https://github.com/newaetech/chipwhisperer-jupyter), distributed under the open-source GPL license. You can distribute and modify this material (even for commercial trainings), provided you maintain references to this repository and the original authors, and also offer your derived material under the same conditions.


ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

All other product names, logos, and brands are property of their respective owners.

The software is provided 'as is', without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.